# Segmentation and Clustering Neighborhoods of Toronto

Notebook to explore and cluster Neighborhoods in Toronto for the IBM Applied Data Science Capstone

# Part 1

### Setup the required dependencies.

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

from bs4 import BeautifulSoup

print('Done!')

Done!


### Scrape the Wikipedia Page

In [2]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_result = requests.get(wikipedia_url)

### Parse the results

In [3]:
# Parse the resulting HTML
soup = BeautifulSoup(wikipedia_result.text, 'lxml')

# Find the table
wikipedia_table = soup.body.table.tbody
wikipedia_table_rows = wikipedia_table.find_all('tr')

# Convert the table to an array
table_results = []
for tr in wikipedia_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        table_results.append(row)

# Convert the array to a Pandas DataFrame
df = pd.DataFrame(table_results, columns=["Postalcode", "Borough", "Neighborhood"])
df.head(10)

Postalcode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M5A  Downtown Toronto       Regent Park
6        M6A        North York  Lawrence Heights
7        M6A        North York    Lawrence Manor
8        M7A      Queen's Park      Not assigned
9        M8A      Not assigned      Not assigned

### Clean the data

In [4]:
# Drop rows where Borough is not assigned
sorted_df = df[df['Borough'] != 'Not assigned']

# Sort rows by first the Postal code, then Borough
sorted_df = sorted_df.sort_values(by=['Postalcode', 'Borough']).reset_index(drop=True)

sorted_df.head(10)

Postalcode      Borough    Neighborhood
0        M1B  Scarborough           Rouge
1        M1B  Scarborough         Malvern
2        M1C  Scarborough  Highland Creek
3        M1C  Scarborough      Rouge Hill
4        M1C  Scarborough      Port Union
5        M1E  Scarborough       Guildwood
6        M1E  Scarborough     Morningside
7        M1E  Scarborough       West Hill
8        M1G  Scarborough          Woburn
9        M1H  Scarborough       Cedarbrae

### Group Neighborhoods by their Postal Codes

In [5]:
# Multiple Neighborhoods can exist in each Postal Code. They will be separated by a comma
grouped_df = sorted_df.groupby('Postalcode').agg({'Borough': 'first', 'Neighborhood': ', '.join}).reset_index()

grouped_df.head(10)

Postalcode      Borough                                     Neighborhood
0        M1B  Scarborough                                   Rouge, Malvern
1        M1C  Scarborough           Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough                Guildwood, Morningside, West Hill
3        M1G  Scarborough                                           Woburn
4        M1H  Scarborough                                        Cedarbrae
5        M1J  Scarborough                              Scarborough Village
6        M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park
7        M1L  Scarborough                  Clairlea, Golden Mile, Oakridge
8        M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West
9        M1N  Scarborough                      Birch Cliff, Cliffside West

### Find Neighborhoods that are not assigned

In [6]:
grouped_df[grouped_df['Neighborhood'] == 'Not assigned']

Postalcode       Borough  Neighborhood
85        M7A  Queen's Park  Not assigned

### Replace not assigned Neighborhoods with the Borough name

In [7]:
# Find Neighborhoods that are not assigned
# Use the Borough name as the Neighborhood name where Neighborhood name is not assigned
grouped_df.loc[grouped_df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = grouped_df['Borough']

grouped_df

Postalcode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North York   
27         M3C        North York   
28         M3H        North York   
29         M3J        North York   
30         M3K        North York   
31         M3L        North York   
32         M3M        North York   
33         M3N        North York   
34         M4A        North York   
35         M4B         East York   
36         M4C         East York   
37         M4E      East Toronto   
38         M4G         East York   
39         M4H         East York   
40         M4J         East York   
41         M4K      East Toronto   
42         M4L      East Toronto   
43         M4M      East Toronto   
44         M4N   Central Toronto   
45         M4P   Central Toronto   
46         M4R   Central Toronto   
47         M4S   Central Toronto   
48         M4T   Central Toronto   
49         M4V   Central Toronto   
50         M4W  Downtown Toronto   
51         M4X  Downtown Toronto   
52         M4Y  Downtown Toronto   
53         M5A  Downtown Toronto   
54         M5B  Downtown Toronto   
55         M5C  Downtown Toronto   
56         M5E  Downtown Toronto   
57         M5G  Downtown Toronto   
58         M5H  Downtown Toronto   
59         M5J  Downtown Toronto   
60         M5K  Downtown Toronto   
61         M5L  Downtown Toronto   
62         M5M        North York   
63         M5N   Central Toronto   
64         M5P   Central Toronto   
65         M5R   Central Toronto   
66         M5S  Downtown Toronto   
67         M5T  Downtown Toronto   
68         M5V  Downtown Toronto   
69         M5W  Downtown Toronto   
70         M5X  Downtown Toronto   
71         M6A        North York   
72         M6B        North York   
73         M6C              York   
74         M6E              York   
75         M6G  Downtown Toronto   
76         M6H      West Toronto   
77         M6J      West Toronto   
78         M6K      West Toronto   
79         M6L        North York   
80         M6M              York   
81         M6N              York   
82         M6P      West Toronto   
83         M6R      West Toronto   
84         M6S      West Toronto   
85         M7A      Queen's Park   
86         M7R       Mississauga   
87         M7Y      East Toronto   
88         M8V         Etobicoke   
89         M8W         Etobicoke   
90         M8X         Etobicoke   
91         M8Y         Etobicoke   
92         M8Z         Etobicoke   
93         M9A         Etobicoke   
94         M9B         Etobicoke   
95         M9C         Etobicoke   
96         M9L        North York   
97         M9M        North York   
98         M9N              York   
99         M9P         Etobicoke   
100        M9R         Etobicoke   
101        M9V         Etobicoke   
102        M9W         Etobicoke   

                                          Neighborhood  
0                                       Rouge, Malvern  
1               Highland Creek, Rouge Hill, Port Union  
2                    Guildwood, Morningside, West Hill  
3                              

### Check the shape of the Toronto DataFrame

In [8]:
grouped_df.shape

(103, 3)

# Part 2

### Load Geospatial data and merge with the Toronto DataFrame

In [9]:
# Load the geo data
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')

# Rename the Postalcode column
geo_data.rename(columns = {'Postal Code': 'Postalcode'}, inplace=True)

# Merge the two DataFrames
grouped_df = pd.merge(grouped_df, geo_data, on='Postalcode')

grouped_df

Postalcode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North York   
27         M3C        North York   
28         M3H        North York   
29         M3J        North York   
30         M3K        North York   
31         M3L        North York   
32         M3M        North York   
33         M3N        North York   
34         M4A        North York   
35         M4B         East York   
36         M4C         East York   
37         M4E      East Toronto   
38         M4G         East York   
39         M4H         East York   
40         M4J         East York   
41         M4K      East Toronto   
42         M4L      East Toronto   
43         M4M      East Toronto   
44         M4N   Central Toronto   
45         M4P   Central Toronto   
46         M4R   Central Toronto   
47         M4S   Central Toronto   
48         M4T   Central Toronto   
49         M4V   Central Toronto   
50         M4W  Downtown Toronto   
51         M4X  Downtown Toronto   
52         M4Y  Downtown Toronto   
53         M5A  Downtown Toronto   
54         M5B  Downtown Toronto   
55         M5C  Downtown Toronto   
56         M5E  Downtown Toronto   
57         M5G  Downtown Toronto   
58         M5H  Downtown Toronto   
59         M5J  Downtown Toronto   
60         M5K  Downtown Toronto   
61         M5L  Downtown Toronto   
62         M5M        North York   
63         M5N   Central Toronto   
64         M5P   Central Toronto   
65         M5R   Central Toronto   
66         M5S  Downtown Toronto   
67         M5T  Downtown Toronto   
68         M5V  Downtown Toronto   
69         M5W  Downtown Toronto   
70         M5X  Downtown Toronto   
71         M6A        North York   
72         M6B        North York   
73         M6C              York   
74         M6E              York   
75         M6G  Downtown Toronto   
76         M6H      West Toronto   
77         M6J      West Toronto   
78         M6K      West Toronto   
79         M6L        North York   
80         M6M              York   
81         M6N              York   
82         M6P      West Toronto   
83         M6R      West Toronto   
84         M6S      West Toronto   
85         M7A      Queen's Park   
86         M7R       Mississauga   
87         M7Y      East Toronto   
88         M8V         Etobicoke   
89         M8W         Etobicoke   
90         M8X         Etobicoke   
91         M8Y         Etobicoke   
92         M8Z         Etobicoke   
93         M9A         Etobicoke   
94         M9B         Etobicoke   
95         M9C         Etobicoke   
96         M9L        North York   
97         M9M        North York   
98         M9N              York   
99         M9P         Etobicoke   
100        M9R         Etobicoke   
101        M9V         Etobicoke   
102        M9W         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                       Rouge, Malvern  43.806686 -79.194353  
1               Highland Creek, Rouge Hill, Port Union  43.784535 -79.160497  
2                    G

# Part 3

### Create a new DataFrame of only Boroughs which contain Toronto

In [10]:
toronto_df = grouped_df[grouped_df.Borough.str.contains('Toronto')]
toronto_df.reset_index(inplace=True)

### Get the coordinates of Toronto

In [11]:
# Address of Toronto
address = 'Toronto, Ontario, Canada'

# Geo locator
geolocator = Nominatim(user_agent="to_explorer")

# Get the lat and long of Toronto
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# Print the results
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto

In [12]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### Connect to Foursqaure

In [13]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605' # Foursquare API version

# credentials removed for public version

### Let's explore the first Neighborhood in the DataFrame

In [14]:
# Get the values of the first row
first = toronto_df.loc[0, 'Neighborhood']
first_lat = toronto_df.loc[0, 'Latitude']
first_lon = toronto_df.loc[0, 'Longitude']

# Print out the results
print('The coordinates of "{}" are lat: {}, long: {}'.format(first, first_lat, first_lon))

The coordinates of "The Beaches" are lat: 43.67635739999999, long: -79.2930312


### Get the top 100 venues within a 500 meter radius of the first Neighborhood

In [15]:
# Radius of our search
radius = 500

# Limit the number of venues
LIMIT = 100

# Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_lat, 
    first_lon, 
    radius, 
    LIMIT)

# Get the result in JSON format
result = requests.get(url).json()

result

{'meta': {'code': 200, 'requestId': '5cb27354f594df78c7172f43'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678879,
          'lng': -79.297734}],
        'distance': 471,
        'postalCode': 'M4E 0B8',
   

### Build a function to extract the venue category

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the response data

In [17]:
venues = json_normalize(result['response']['groups'][0]['items'])

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:, filtered_columns]

# filter the category for each row
venues['venue.categories'] = venues.apply(get_category_type, axis=1)

# clean columns
venues.columns = [col.split(".")[-1] for col in venues.columns]

venues.head()

name         categories        lat        lng
0  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
1                 Grover Pub and Grub                Pub  43.679181 -79.297215
2                           Starbucks        Coffee Shop  43.678798 -79.298045
3                       Upper Beaches       Neighborhood  43.680563 -79.292869

### Verify the number of venues returned

In [18]:
print('{} venues were returned by Foursquare.'.format(venues.shape[0]))

4 venues were returned by Foursquare.


### Build a function to apply the same method to all Neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Use the gerNearbyVenues() function to get venues for all Neighborhoods

In [20]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

### Take a quick look at the data

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1709, 7)


Neighborhood  Neighborhood Latitude  \
0                   The Beaches              43.676357   
1                   The Beaches              43.676357   
2                   The Beaches              43.676357   
3                   The Beaches              43.676357   
4  The Danforth West, Riverdale              43.679557   

   Neighborhood Longitude                               Venue  Venue Latitude  \
0              -79.293031  The Big Carrot Natural Food Market       43.678879   
1              -79.293031                 Grover Pub and Grub       43.679181   
2              -79.293031                           Starbucks       43.678798   
3              -79.293031                       Upper Beaches       43.680563   
4              -79.352188                            Pantheon       43.677621   

   Venue Longitude     Venue Category  
0       -79.297734  Health Food Store  
1       -79.297215                Pub  
2       -79.298045        Coffee Shop  
3       -79.292869       Neighborhood  
4       -79.351434   Greek Restaurant

### Let's see how many venues each Neighborhood has

In [22]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Berczy Park                                                            55   
Brockton, Exhibition Place, Parkdale Village                           20   
Business Reply Mail Processing Centre 969 Eastern                      17   
CN Tower, Bathurst Quay, Island airport, Harbou...                     14   
Cabbagetown, St. James Town                                            43   
Central Bay Street                                                     84   
Chinatown, Grange Park, Kensington Market                             100   
Christie                                                               16   
Church and Wellesley                                                   89   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             37   
Davisville North                                                       11   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     15   
Design Exchange, Toronto Dominion Centre                              100   
Dovercourt Village, Dufferin                                           19   
First Canadian Place, Underground city                                100   
Forest Hill North, Forest Hill West                                     4   
Harbord, University of Toronto                                         36   
Harbourfront East, Toronto Islands, Union Station                     100   
Harbourfront, Regent Park                                              48   
High Park, The Junction South                                          25   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               64   
Moore Park, Summerhill East                                             4   
North Toronto West                                                     19   
Parkdale, Roncesvalles                                                 15   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     39   
Ryerson, Garden District                                              100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                        97   
Studio District                                                        37   
The Annex, North Midtown, Yorkville                                    25   
The Beaches                                                             4   
The Beaches West, India Bazaar                                         22   
The Danforth West, Riverdale                                           42   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                               100   
Berczy Park                                                             55   
Brockton, Exhibition Place, Parkdale Village                            20   
Business Reply Mail Processing Centre 969 Eastern                       17   
CN Tower, Bathurst Quay, Island airport, Harbou...                      14   
Cabbagetown, St. James Town                                             43   
Central Bay Street                                                      84   
Chinatown, Grange Park, Kensington Market                              100   
Christie                                                                16   
Church and Wellesley                                                    89   
Commerce Court, Victoria Hotel     

### See how many unique venue cetegories there are in the data

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


### Now time to analyze the data

In [24]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add the Neighborhood columns back in
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# Move neighborhood column to the first column
columns = toronto_onehot.columns.tolist()
columns.insert(0, columns.pop(columns.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns=columns)

toronto_onehot.head()

Neighborhood  Accessories Store  Adult Boutique  \
0                   The Beaches                  0               0   
1                   The Beaches                  0               0   
2                   The Beaches                  0               0   
3                   The Beaches                  0               0   
4  The Danforth West, Riverdale                  0               0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Argentinian Restaurant  Art Gallery  Art Museum  \
0             0         0                       0            0           0   
1             0         0                       0            0           0   
2             0         0                       0            0           0   
3             0         0                       0            0           0   
4             0         0                       0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                    0                 0                   0              0   
1                    0                 0                   0              0   
2                    0                 0                   0              0   
3                    0                 0                   0              0   
4                    0                 0                   0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0          0           0           0       0     0    0                 0   
1          0           0           0       0     0    0                 0   
2          0           0           0       0     0    0                 0   
3          0           0           0       0     0    0                 0   
4          0           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Beer Bar  Beer Store  Belgian Restaurant  \
0                   0      0         0           0                   0   
1                   0      0         0           0                   0   
2                   0      0         0           0                   0   
3                   0      0         0           0                   0   
4                   0      0         0           0                   0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bubble Tea Shop  Building  Burger Joint  \
0               0        0                0         0             0   
1               0        0                0         0             0   
2               0        0                0         0             0   
3               0        0                0         0             0   
4               0        0                0         0             0   

   Burrito Place  Bus Line

### Now lets group the data by frequency of Venue Category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Accessories Store  \
0                            Adelaide, King, Richmond               0.01   
1                                         Berczy Park               0.00   
2        Brockton, Exhibition Place, Parkdale Village               0.00   
3   Business Reply Mail Processing Centre 969 Eastern               0.00   
4   CN Tower, Bathurst Quay, Island airport, Harbo...               0.00   
5                         Cabbagetown, St. James Town               0.00   
6                                  Central Bay Street               0.00   
7           Chinatown, Grange Park, Kensington Market               0.00   
8                                            Christie               0.00   
9                                Church and Wellesley               0.00   
10                     Commerce Court, Victoria Hotel               0.00   
11                                         Davisville               0.00   
12                                   Davisville North               0.00   
13  Deer Park, Forest Hill SE, Rathnelly, South Hi...               0.00   
14           Design Exchange, Toronto Dominion Centre               0.00   
15                       Dovercourt Village, Dufferin               0.00   
16             First Canadian Place, Underground city               0.00   
17                Forest Hill North, Forest Hill West               0.00   
18                     Harbord, University of Toronto               0.00   
19  Harbourfront East, Toronto Islands, Union Station               0.00   
20                          Harbourfront, Regent Park               0.00   
21                      High Park, The Junction South               0.00   
22                                      Lawrence Park               0.00   
23                           Little Portugal, Trinity               0.00   
24                        Moore Park, Summerhill East               0.00   
25                                 North Toronto West               0.00   
26                             Parkdale, Roncesvalles               0.00   
27                                           Rosedale               0.00   
28                                           Roselawn               0.00   
29                                 Runnymede, Swansea               0.00   
30                           Ryerson, Garden District               0.00   
31                                     St. James Town               0.01   
32                    Stn A PO Boxes 25 The Esplanade               0.00   
33                                    Studio District               0.00   
34                The Annex, North Midtown, Yorkville               0.00   
35                                        The Beaches               0.00   
36                     The Beaches West, India Bazaar               0.00   
37                       The Danforth West, Riverdale               0.00   

    Adult Boutique  Afghan Restaurant   Airport  Airport Food Court  \
0         0.000000           0.000000  0.000000            0.000000   
1         0.000000           0.000000  0.000000            0.000000   
2         0.000000           0.000000  0.000000            0.000000   
3         0.000000           0.000000  0.000000            0.000000   
4         0.000000           0.000000  0.071429            0.071429   
5         0.000000           0.000000  0.000000            0.000000   
6         0.000000           0.000000  0.000000            0.000000   
7         0.000000           0.000000  0.000000            0.000000   
8         0.000000           0.000000  0.000000            0.000000   
9         0.011236           0.011236  0.000000            0.000000   
10        0.000000           0.000000  0.000000            0.000000   
11        0.000000           0.000000  0.000000            0.000000   
12        0.000000           0.000000  0.000000            0.000000   
13        0.000000           0.000000  0.000000            0.000000   
14        0

### Print the top 5 venue cetegories for each Neighborhood

In [26]:
# Number of venues to load
num_top_venues = 5

# Loop through each Neighborhood
for hood in toronto_grouped['Neighborhood']:
    
    # Print the Neighborhood name
    print("----"+hood+"----")
    
    # Create a temp array for the current Neighborhood
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    
    # Add venue and frequency columns
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    
    # Convert the frequency to a float with 2 decimal places
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
    # Print the results
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2  Farmers Market  0.04
3     Cheese Shop  0.04
4      Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.10
1            Café  0.10
2     Coffee Shop  0.10
3             Gym  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1       Auto Workshop  0.06
2          Restaurant  0.06
3                 Spa  0.06
4         Pizza Place  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2  Airport Terminal  0.14
3  

### Put this data into a Pandas DataFrame

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village        Breakfast Spot   
3  Business Reply Mail Processing Centre 969 Eastern    Light Rail Station   
4  CN Tower, Bathurst Quay, Island airport, Harbo...       Airport Service   

  2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0            Steakhouse       Thai Restaurant                   Café   
1          Cocktail Bar                  Café            Cheese Shop   
2                  Café           Coffee Shop  Performing Arts Venue   
3           Pizza Place         Auto Workshop             Comic Shop   
4      Airport Terminal        Airport Lounge        Harbor / Marina   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                   Bar                Bakery   American Restaurant   
1                Bakery    Seafood Restaurant            Steakhouse   
2          Climbing Gym         Burrito Place               Stadium   
3      Recording Studio            Restaurant         Burrito Place   
4          Airport Gate      Sculpture Garden                 Plane   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0      Sushi Restaurant          Burger Joint             Restaurant  
1                   Pub            Restaurant         Farmers Market  
2                   Bar            Restaurant   Caribbean Restaurant  
3               Brewery            Skate Park             Smoke Shop  
4    Airport Food Court               Airport          Boat or Ferry

### Clustering the data with K-Means

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       4, 0, 3, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Add the clusters to the DataFrame

In [30]:
# Add clusters column
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Merge with the original DataFrame
toronto_merged = toronto_df.drop('index', axis=1)

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

Postalcode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0     Health Food Store           Coffee Shop   
1 -79.352188               0      Greek Restaurant           Coffee Shop   
2 -79.315572               0        Sandwich Place          Liquor Store   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               4                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Pub    Dim Sum Restaurant        Farmers Market   
1        Ice Cream Shop    Italian Restaurant         Grocery Store   
2        Ice Cream Shop          Burger Joint         Burrito Place   
3             Gastropub                Bakery    Italian Restaurant   
4              Bus Line           Yoga Studio        Discount Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0    Falafel Restaurant           Event Space  Ethiopian Restaurant   
1             Bookstore               Brewery       Bubble Tea Shop   
2  Fast Food Restaurant    Light Rail Station     Fish & Chips Shop   
3   American Restaurant           Yoga Studio             Bookstore   
4  Fast Food Restaurant        Farmers Market    Falafel Restaurant   

  9th Most Common Venue       10th Most Common Venue  
0     Electronics Store  Eastern European Restaurant  
1                  Café                   Restaurant  
2            Steakhouse            Food & Drink Shop  
3               Brewery           Seafood Restaurant  
4           Event Space         Ethiopian Restaurant

### Visualize the clusters on a map

In [31]:
from matplotlib import cm
import branca.colormap as branca

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters